In [ ]:
using Plots
using FFTW

In [ ]:
# reverse a signal about the first element
flip(x) = x[-(0:length(x)-1) .& (length(x)-1) .+ 1]

linphase(offs, n) = cispi.(offs*-2(0:n-1)/n)
shift(x, offs) = x.*linphase(offs, length(x))

# FFT Identities

$x$ is a signal with elements from $(1, n]$: $\{x_n\}$

$F(x) = X$

## Linearity

Addition and scalar multiplication work as expected.

$F(ax + by) = aF(x) + bF(y)$

## Reversing Time and Frequency

Reversing time reverses frequency.

`flip(fft(x)) = fft(flip(x))`

### Conjugating has a similar effect

Conjugate in time is equal to the conjugate of the reverse in freq.

`fft(conj(x)) = conj(flip(fft(x))) = conj(fft(flip(x)))`

Conjugate in freq is equal to the conjugate of the reverse in time.

`conj(fft(x)) = flip(fft(conj(x)))) = fft(flip(conj(x))))`

<!-- Swapping re/im acts similar to conjugation. (Note: I feel like there was a better identity to use here...)

`conj(fft(x)*im) = flip(conj(fft((x)))*im)` -->

## Almost it's own inverse

Same as FFT, but flipped and scaled.

$F^{-1}(x) = \frac{F(x)_{N-n}}{N} = \frac{F(x_{N-n})}{N}$

Since flipping and conjugation are related, you can use that too.

$F^{-1}(x) = \frac{F(x^*)^*}{N}$

Also you can swap re/im before and after. (conjugate * i)

$F^{-1}(x) = \frac{F(x^*i)^*i}{N}$

## Real/Imaginary Valued FFTs

Real valued FFTs have a symmetric result. ex:

$F(x) = [a, b, c, d, c^*, b^*, a^*]$

## Zero padding

Padding out a signal with zeros doesn't add any addtional information. It will increase the frequency resolution of the fft without adding any addition frequencies to it.

## Repetition and Sequences

If you have a signal $x$, it will have an FFT of the form:

$F(x) = \{a, b, c, d\}$

If you repeat the signal, the even values will all be zeros. This is because the two halves are the same except for the phase. The even values are rotated by 180° so they cancel out.

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, d, 0\}$

If $x$ is real, then you get the usual symmetry as well:

$F(\{x, x\}) = 2\{a, 0, b, 0, c, 0, b^*, 0\}$

$F(\{x; reverse(x)\}) = \{e, f, g, h, 0, -h, -g, -f\}$

In [ ]:
N = 8
x, y = rand(0:9, N) + rand(0:9, N)im, rand(0:9, N) + rand(0:9, N)im
X, Y = fft(x), fft(y)
0

In [ ]:
f() = fft(reverse(x))
g() = reverse(fft(x))
round.(f() - g(); digits = 10)

plot(
    plot([real(f()), real(g())]),
    plot([imag(f()), imag(g())]),
    plot([abs.(f()), abs.(g())]),
    label = ["f" "g"]
)

In [ ]:
plot()

xx = [x; reverse(x)]

fft((xx)).*linphase(0.5, 2N)

In [ ]:
z = zeros(ComplexF64, 2N)
z[2:2:2N] .= real(x)
Z = fft(z)

plot(
    plot(real(Z)),
    plot(imag(Z)),
    plot(abs.(Z)),
    label = ["f" "g"]
)
round.(Z; digits=3)

Z[0N + 1:1N] + Z[1N + 1:2N]
# [x; -x] sequence?

In [ ]:
N = 8
# x = Array(1:N)
# x = Array(1:N).^2
x = rand(-9:9, N) + rand(-9:9, N)*im
# x = cos.((0:N-1).*(6π/N))
# plot(x)

y = fft(x)
# plot([x, abs.(y)])
round.([x y]; digits=2)


# Calculating a complex valued FFT

In [ ]:
# calculate FFT of odd elements
xo = Array(x)
xo[2:2:N] .= 0
yo = fft(xo)

# calculate FFT of even elements
xe = Array(x)
xe[1:2:N] .= 0
ye = fft(xe)

# FFT is linear, you can add the results
round.([yo ye yo + ye]; digits=2)

The FFT of a sequence with even elements 0s is the FFT of the odd elements repeated twice.

The FFT of a sequence with odd elements 0s is the FFT of the evens phase shifted by half a sample and repeated twice with the second repetition negated.

In [ ]:
# fft of evens and odds without zeros inserted
phase = cispi.(-(0:N÷2-1)./(N÷2))
yo = fft(x[1:2:N])
ye = fft(x[2:2:N]).*phase

# add them to get the full fft
round.([yo ye yo + ye; yo -ye yo - ye;]; digits=2)

# Real and Imaginary Valued FFTs

In [ ]:
# calculate FFT of just real values
yre = fft(real(x))
yim = fft(imag(x)im)

# FFT is linear, you can add the results
round.([yre yim yre + yim]; digits=2)

In [ ]:
# works the other way too if you have the FFT
y2 =  conj(flip(y))
yre = (y + y2)/2
yim = (y - y2)/2
round.([yre yim]; digits=2)

# Real Valued FFTs

In [ ]:
x = real(x)
xo = x[1:2:N]
xe = x[2:2:N]
round.([x fft(x)]; digits=2)

In [ ]:
x2 = x[1:2:N] + x[2:2:N]im
y = fft(x2)
y2 = conj(flip(y))
yo = (y + y2)/2
ye = (y - y2)/2 * -im.*phase

round.(yo + ye; digits=2)

# Calculating DCTs via the FFT

## DCT-II

In [ ]:
dct2(x) = real(fft([x; reverse(x)])[1:N].*linphase(0.25, N))
dct2(x)

In [ ]:
foo = [x; zeros(N)]
# bar = [zeros(N); reverse(x)]
bar = reverse(foo)
(fft(foo) + fft(bar)).*linphase(0.5, 2N)

## DCT-III

In [ ]:
dct3(x) = real(fft([x; 0; -reverse(x); -x[2:N]; 0; reverse(x[2:N])])[2:2:2N])/2
dct3(dct2(x))/2N - x

## DCT-IV

In [ ]:
dct4(x) = real(shift(fft([x; -reverse(x); -x; reverse(x)]), 0.5)[2:2:2N])

norm4 = sqrt(2)/N
# dct4(dct4(x))*norm4^2 - 2x
round.(dct4(x); digits=2)

In [ ]:
foo = [x; -reverse(x); -x; reverse(x)].*linphase(1, 4N)
bar = fft(foo).*cispi.((1:4N)*-0.125*2/N)
round.(bar; digits=2)

# thoughts: this produces an fft([a -a]) = [b, -b] pattern in the output, need to find an identity here